In [7]:
import pandas as pd

# Load each CSV file into a separate DataFrame
file_path1 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling bengali\urls_and_articles_summary_bengali.csv"
file_path2 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling gujarati\urls_and_articles_summary_gujarati.csv"
file_path3 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling hindi\urls_and_articles_summary_hindi.csv"
file_path4 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling marathi\urls_and_articles_summary_marathi.csv"
file_path5 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling tamil\urls_and_articles_summary_tamil.csv"
file_path6 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling telugu\urls_and_articles_summary_telugu.csv"

# Load each file and store in a separate DataFrame
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)
df4 = pd.read_csv(file_path4)
df5 = pd.read_csv(file_path5)
df6 = pd.read_csv(file_path6)

# Preview the data for each file to confirm the "Article" column is loaded correctly
print("File 1 Preview:")
print(df1.head())

print("File 2 Preview:")
print(df2.head())

print("File 3 Preview:")
print(df3.head())

print("File 4 Preview:")
print(df4.head())

print("File 5 Preview:")
print(df5.head())

print("File 6 Preview:")
print(df6.head())



File 1 Preview:
                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
File 2 Preview:
            

In [5]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API'  # Replace with your actual API key

# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Bengali language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df1...")
generated_summaries, rouge_scores = process_dataframe(df1)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df1['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_bengali.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")



Processing df1...
Generated Summary for Article: কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের এক দ...
মহারাষ্ট্রের কোলাপুরের এক দম্পতির বিয়ে ২০০৯ সালে হয়, কিন্তু দীর্ঘ ন'বছর ধরে তাদের মধ্যে শারীরিক সম্পর্ক হয়নি। স্ত্রী আদালতে অভিযোগ করেন যে তাকে ধোঁকা দিয়ে বিয়েতে রাজি করানো হয়েছে। মুম্বাই হাইকোর্ট জানায়, শারীরিক সম্পর্কের অভাবের কারণে বিয়েটি অর্থহীন। বিচারপতি মৃদুলা ভাটকর বলেন, বিয়ের একটি প্রধান উপাদান হল স্বামী-স্ত্রীর মধ্যে নিয়মিত শারীরিক সম্পর্ক, যা অনুপস্থিত থাকলে বিয়ের উদ্দেশ্য ব্যর্থ হয়। স্বামীর দাবি ছিল যে তাদের মধ্যে সম্পর্ক
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.21739130434782608, 'p': 0.08620689655172414, 'f': 0.12345678605700364}, 'rouge-2': {'r': 0.09090909090909091, 'p': 0.03125, 'f': 0.046511624099513565}, 'rouge-l': {'r': 0.21739130434782608, 'p': 0.08620689655172414, 'f': 0.12345678605700364}}
Generated Summary for Article: বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার ব

In [5]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API'
# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Hindi language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

# Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df2...")
generated_summaries, rouge_scores = process_dataframe(df2)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df2['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_hindi.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df2...
Generated Summary for Article: શશી થરૂરની ટિપ્પણીથી સોશિયલ મીડિયામાં ચર્ચા થરૂરે ...
શશી થરૂરે 'બેંગલુરુ લિટરેચર ફૅસ્ટ'માં આરએસએસના સૂત્રનો ઉલ્લેખ કરીને વડા પ્રધાન નરેન્દ્ર મોદીને "શિવલિંગ પર બેઠેલા વીંછી" સાથે તુલના કરી હતી, જે પરિસ્થિતિને દૂર કરવી મુશ્કેલ છે. તેમના આ નિવેદનથી સોશિયલ મીડિયામાં વિવાદ ઊભો થયો છે, અને ભાજપના નેતાઓએ આ પર પ્રતિક્રિયા આપી છે. કોંગ્રેસે થરૂરના નિવેદનથી પોતાને અલગ કરી લીધી છે, અને થરૂરે જણાવ્યું છે કે આ તેમનું વ્યક્તિગત મત છે.
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.42857142857142855, 'p': 0.11764705882352941, 'f': 0.184615381235503}, 'rouge-2': {'r': 0.15384615384615385, 'p': 0.03333333333333333, 'f': 0.05479451762056687}, 'rouge-l': {'r': 0.42857142857142855, 'p': 0.11764705882352941, 'f': 0.184615381235503}}
Generated Summary for Article: સામાચાર એજન્સી એએનઆઈ પ્રમાણે આગ લાગવાની આ ઘટના શૉર...
આ લેખમાં જણાવાયું છે કે ઉત્તરાખંડમાં આગ લાગવાની ઘટના શૉર્ટ 

In [9]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API KEY'  # Replace with your actual API key

# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Hindi language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()



# Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df3...")
generated_summaries, rouge_scores = process_dataframe(df3)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df3['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_hindi.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df3...
Generated Summary for Article: ओलंपिक खेल की संयोजक समिति के अधिकारियों ने माना ह...
ओलंपिक खेलों की संयोजक समिति के अधिकारियों ने स्वीकार किया है कि स्टेडियमों में दर्शकों की कमी के कारण उन्हें स्वयंसेवकों की मदद लेनी पड़ रही है। वरिष्ठ अधिकारी वांग वे ने बताया कि बारिश और गर्म मौसम जैसे कारणों से दर्शक केवल कुछ विशेष खेलों के लिए ही आते हैं, जबकि उनके पास पूरे सेशन के टिकट होते हैं। कई मैचों में दर्शकों की उपस्थिति कम रही, जैसे कि बास्केट बॉल और जूडो में। खाली सीटों की एक वजह प्रायोजकों के लिए सुरक्षित सीटों का न भरना भी हो सकता है।
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.4444444444444444, 'p': 0.2191780821917808, 'f': 0.29357797722750617}, 'rouge-2': {'r': 0.09090909090909091, 'p': 0.045454545454545456, 'f': 0.060606056161616496}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.1643835616438356, 'f': 0.22018348181466216}}
Generated Summary for Article: दोनों ने भारत के वीनू मनकड औ

In [11]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API KEY'  # Replace with your actual API key

# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Marathi language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()



# Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df4...")
generated_summaries, rouge_scores = process_dataframe(df4)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df4['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_marathi.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df4...
Generated Summary for Article: लवकर केस पांढरे होत असतील तर त्याला वैद्यकीय भाषेत...
केस लवकर पांढरे होणे म्हणजे 'केनाइटिस'. वयाच्या 20 वर्षांपूर्वीच केस पांढरे झाले असल्यास हे असू शकते. यामागे अनुवांशिकता, कमी पोषण, हार्मोन्सचे असंतुलन किंवा हिमोग्लोबीन कमी असणे कारणीभूत असू शकते. यावर इलाज करणे अवघड असून औषधं आणि शँपूचा उपयोग कमी आहे. बायोटिनयुक्त आहार घेणे, केमिकल्सचा वापर टाळणे आणि जास्त शँपूचा वापर न करणे हे उपाय आहेत.
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.2631578947368421, 'p': 0.11363636363636363, 'f': 0.1587301545175108}, 'rouge-2': {'r': 0.05555555555555555, 'p': 0.018867924528301886, 'f': 0.028169010299544252}, 'rouge-l': {'r': 0.2631578947368421, 'p': 0.11363636363636363, 'f': 0.1587301545175108}}
Generated Summary for Article: स्टीव्हन स्मिथने वेगवान शतक झळकावलं. सिडनी क्रिकेट...
सिडनी क्रिकेट ग्राऊंडवर स्टीव्हन स्मिथने वेगवान शतक झळकावले. या मॅचमध्ये एक भारतीय माणस

In [13]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API KEY'

# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Tamil language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()



# Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df5...")
generated_summaries, rouge_scores = process_dataframe(df5)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df5['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_Tamil.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df5...
Generated Summary for Article: சித்தரிப்புக்காக வெனிசுவேலா நாட்டில் ஏறத்தாழ 50 ஆய...
வெனிசுவேலாவில் 50,000 வாக்கு இயந்திரங்கள் மற்றும் 582 கணினிகள் தீக்கிரையானதாக தேர்தல் ஆணையம் தெரிவித்துள்ளது. கராகஸ் அருகே ஏற்பட்ட தீ விபத்தியில் இவை எரிந்து சாம்பலானதாகும். தேர்தல் தள்ளிப்போகுமா என்பது குறித்து அதிகாரிகள் தகவல் வழங்கவில்லை. தீ விபத்திக்கான காரணம் இன்னும் தெரியவில்லை.
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.3333333333333333, 'p': 0.10344827586206896, 'f': 0.15789473322714692}, 'rouge-2': {'r': 0.125, 'p': 0.03333333333333333, 'f': 0.05263157562326891}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.10344827586206896, 'f': 0.15789473322714692}}
Generated Summary for Article: எங்களின் போராட்டம் மக்களின் எதார்த்த வாழ்க்கையை பா...
கமல்ஹாசன், மக்களின் வாழ்க்கையை பாதிக்காமல் போராட வேண்டும் என கூறியுள்ளார். அவர், அரசியலில் ரஜினிகாந்தை எதிர்க்க வேண்டிய சூழல் வந்தால் எதிர்ப்பேன் என்று தெரிவி

In [15]:
import openai
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your OpenAI API key
openai.api_key = 'API KEY'  # Replace with your actual API key

# Function to summarize articles using OpenAI's API
def summarize_article(article):
    try:
        prompt = f"""
        Summarize the following article briefly.
        Only use Telugu language in summary
        Article:
        {article}
        Summary:
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust based on the desired length of the summary
            temperature=0.3  # Adjust creativity of the response
        )
        
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

# Process the specified DataFrame (df1)
def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()



# Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]  # Adjust to your reference column
                    score = rouge.get_scores(summary, reference_summary, avg=True)
                    rouge_scores.append(score)
                    print(f"ROUGE Score for the summary: {score}")

                else:
                    all_summaries.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df6...")
generated_summaries, rouge_scores = process_dataframe(df6)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df6['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_Telugu.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df6...
Generated Summary for Article: పాకిస్తాన్, చైనా, నేపాల్‌లతో భారత సరిహద్దు వివాదాల...
భారతదేశం పాకిస్తాన్, చైనా, నేపాల్ వంటి దేశాలతో 15,106.7 కి.మీ.ల సరిహద్దు కలిగి ఉంది. ఈ సరిహద్దులలో ఎల్ఓసీ (నియంత్రణ రేఖ), ఎల్ఏసీ (వాస్తవాధీన రేఖ) మరియు అంతర్జాతీయ సరిహద్దు రేఖలు ఉన్నాయి. భారత్-చైనా మధ్య సరిహద్దు 3,488 కి.మీ.లు పొడవు ఉంది, ఇది మూడు సెక్టార్లుగా విభజించబడింది: పశ్చిమ, మిడిల్, తూర్పు. ఈ సరిహద్దుల
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.08333333333333333, 'p': 0.05, 'f': 0.062499995312500355}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.08333333333333333, 'p': 0.05, 'f': 0.062499995312500355}}
Generated Summary for Article: సుప్రీంకోర్టు నలుగురు సుప్రీంకోర్టు సీనియర్ న్యాయమ...
సుప్రీంకోర్టు నలుగురు సీనియర్ న్యాయమూర్తులు మీడియా సమావేశంలో ప్రధాన న్యాయమూర్తి జస్టిస్ దీపక్ మిశ్రా వ్యవహారశైలి, రోస్టర్ విధానం వంటి అంశాలను చర్చించారు. రాజు రామచంద్రన్ ఈ సమావేశాన్ని "తీవ్రమై